In [153]:
import requests
import secrets
import hashlib
import base64
from urllib.parse import urlencode, quote
from flask import Flask, request, redirect, url_for, session

In [168]:
# Datos de tu aplicación
CLIENT_ID = "01JMMS7DVKKB10JM3KHCMCQ355"
CLIENT_SECRET = "29ccdadaac0c46088f55f75edc11d7c559db8f0c5a8c1f2e79793f7e9f8d1848"
REDIRECT_URI = "https://blank-app-7n69k0rfqzl.streamlit.app"
SCOPES = "user:read channel:read"

app = Flask(__name__)
app.secret_key = secrets.token_urlsafe(32)  # ¡Secreto para la sesión!




def generate_code_verifier():
    """Genera un código verificador aleatorio."""
    return secrets.token_urlsafe(32)

def generate_code_challenge(code_verifier):
    """Genera un código challenge usando PKCE."""
    code_challenge = hashlib.sha256(code_verifier.encode('utf-8')).digest()
    code_challenge = base64.b64encode(code_challenge).decode('utf-8')
    code_challenge = code_challenge.replace('+', '-').replace('/', '_').replace('=', '')
    return code_challenge

# --- Flujo de autorización ---

def get_authorization_url():
    """Construye la URL de autorización."""
    code_verifier = generate_code_verifier()
    code_challenge = generate_code_challenge(code_verifier)
    state = secrets.token_urlsafe(16)  # Genera un valor aleatorio para el estado

    params = {
        "response_type": "code",
        "client_id": CLIENT_ID,
        "redirect_uri": REDIRECT_URI,
        "scope": SCOPES,
        "code_challenge": code_challenge,
        "code_challenge_method": "S256",
        "state": state
    }

    authorization_url = f"https://id.kick.com/oauth/authorize?{urlencode(params)}"
    return authorization_url, code_verifier, state


def get_access_token(authorization_code, code_verifier):
    """Intercambia el código de autorización por un token de acceso."""
    data = {
        "grant_type": "authorization_code",
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
        "redirect_uri": REDIRECT_URI,
        "code_verifier": code_verifier,
        "code": authorization_code
    }       
        

    response = requests.post("https://id.kick.com/oauth/token", data=data)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error al obtener el token: {response.status_code} - {response.text}")
        return None



def refresh_access_token(authorization_code):
    """Refresca el token de acceso usando el refresh token."""
    data = {
        "grant_type": "refresh_token",
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
        "refresh_token": authorization_code,
    }
    response = requests.post("https://id.kick.com/oauth/token", data=data)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error al refrescar el token: {response.status_code} - {response.text}")
        return None

def revoke_token(token, token_hint_type=None):
    """Revoca un token (access token o refresh token)."""
    params = {"token": token}
    if token_hint_type:
        params["token_hint_type"] = token_hint_type
    print(f"https://id.kick.com/oauth/revoke?{urlencode(params)}")
    response = requests.post(f"https://id.kick.com/oauth/revoke?{urlencode(params)}")

    if response.status_code == 200:
        print("Token revocado correctamente.")
    else:
        print(f"Error al revocar el token: {response.status_code} - {response.text}")

#########################################################
@app.route("/callback")
def callback():
    code = request.args.get("code")
    state = request.args.get("state")

    if state != session.get("state"):  # ¡Verificación del estado!
        return "Error: Estado no válido", 400

    if code:
        token_data = get_access_token(code, session.get("code_verifier"))
        if token_data:
            session["access_token"] = token_data["access_token"]
            session["refresh_token"] = token_data["refresh_token"]
            return redirect(url_for("success")) # Redirige a una página de éxito
        else:
            return "Error al obtener el token"
    else:
        return "Error: No se recibió el código de autorización"


@app.route("/success") # Página de éxito
def success():
    access_token = session.get("access_token")
    if access_token:
        response = requests.get(
            "https://api.kick.com/public/v1/categories?q=ga%",
            headers={"Authorization": f"Bearer {access_token}"}, # Usa Bearer token
        )
        data = response.json()
        return f"Datos de la API: {data}"  # Muestra los datos
    else:
        return "No hay token de acceso"

@app.route("/")
def index():
    auth_url, code_verifier, state = get_authorization_url()
    session["code_verifier"] = code_verifier # Guarda code_verifier en la sesión
    session["state"] = state # Guarda state en la sesión
    return f'<a href="{auth_url}">Autorizar con Kick</a>'

if __name__ == "__main__":
    app.run(debug=True, port=5001)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

In [169]:
%tb

SystemExit: 1

In [170]:
from flask import Flask

app = Flask(__name__)

@app.route("/")
def hello():
    return "Hola, mundo!"

if __name__ == "__main__":
    app.run(debug=True, port=5001)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

d:\14. Github\blank-app\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [171]:
%tb

SystemExit: 1

In [139]:
auth_url, code_verifier, state = get_authorization_url()
print(f"Redirige al usuario a esta URL: {auth_url}")

Redirige al usuario a esta URL: https://id.kick.com/oauth/authorize?response_type=code&client_id=01JMMS7DVKKB10JM3KHCMCQ355&redirect_uri=https%3A%2F%2Fblank-app-7n69k0rfqzl.streamlit.app&scope=user%3Aread+channel%3Aread&code_challenge=GHqc0QD3yf1AUsxouKdlEuUAMG3uUyxjtin9DUwvdmA&code_challenge_method=S256&state=lSNmiCShWH6fE5RQOerF_g


In [127]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from urllib.parse import urlparse, parse_qs
import time

In [134]:
def obtener_codigo_autorizacion_con_selenium():
    auth_url, stored_code_verifier, stored_state = get_authorization_url()
    print(1)
    options = webdriver.ChromeOptions()  # Opciones del navegador (puedes usar FirefoxOptions, etc.)
    # options.add_argument("--headless")  # Ejecutar el navegador en segundo plano (opcional)

    driver = webdriver.Chrome(options=options)  # Inicializa el navegador (Chrome en este ejemplo)
    driver.get(auth_url)  # Carga la URL de autorización
    print(2)
    # --- ¡INTERACCIÓN CON LA PÁGINA DE AUTORIZACIÓN! ---
    # Aquí debes encontrar el botón de autorización y hacer clic en él.
    # Esto dependerá de la estructura HTML de la página de autorización de Kick.
    # Ejemplo:
    try:
        time.sleep(4)
        boton_autorizar = driver.find_element(By.XPATH, "//button[contains(text(), 'Allow Access')]") # Contiene el texto "Allow Access"
        boton_autorizar.click()
        time.sleep(1)
    except Exception as e:
        print(f"Error al hacer clic en el botón (XPath - Texto): {e}")
        driver.quit()
        return None, None
    print(3)
    # --- CAPTURA LA URL DE REDIRECCIÓN ---
    redirected_url = driver.current_url
    driver.quit()  # Cierra el navegador
    print(4)
    # --- EXTRAE EL CÓDIGO DE AUTORIZACIÓN Y EL ESTADO ---
    parsed_url = urlparse(redirected_url)
    query_params = parse_qs(parsed_url.query)
    print(5)
    code = query_params.get("code", [None])[0]
    state = query_params.get("state", [None])[0]
    print(f"code: {code}, state: {state}, stored_code_verifier: {stored_code_verifier}")
    return code, state, stored_code_verifier  # Devuelve el código, el estado y el code_verifier

# --- Ejemplo de uso ---
code, state, code_verifier = obtener_codigo_autorizacion_con_selenium()

if code and state:
    # --- ¡VERIFICACIÓN CRUCIAL DEL ESTADO! ---
    # ... (verifica que 'state' coincida con 'stored_state') ...
    print(f"code: {code}, state: {state}")
    # token_data = get_access_token(code, code_verifier)
    # ... (resto del código)
else:
    print("No se pudo obtener el código de autorización con Selenium.")

1
2
Error al hacer clic en el botón (XPath - Texto): Message: no such element: Unable to locate element: {"method":"xpath","selector":"//button[contains(text(), 'Allow Access')]"}
  (Session info: chrome=133.0.6943.127); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6233B6EE5+28773]
	(No symbol) [0x00007FF6233225D0]
	(No symbol) [0x00007FF6231B8FAA]
	(No symbol) [0x00007FF62320F286]
	(No symbol) [0x00007FF62320F4BC]
	(No symbol) [0x00007FF623262A27]
	(No symbol) [0x00007FF62323728F]
	(No symbol) [0x00007FF62325F6F3]
	(No symbol) [0x00007FF623237023]
	(No symbol) [0x00007FF6231FFF5E]
	(No symbol) [0x00007FF6232011E3]
	GetHandleVerifier [0x00007FF62370422D+3490733]
	GetHandleVerifier [0x00007FF62371BA13+3586963]
	GetHandleVerifier [0x00007FF62371144D+3544525]
	GetHandleVerifier [0x00007FF62347C9AA+838442]
	(No symbol) [0x00007FF62332D01F]
	(No symbol

ValueError: not enough values to unpack (expected 3, got 2)

In [151]:
authorization_code  = 'YJBLYZQXODMTNDIWYY0ZYZY1LWFMMJETOGI0MJQZNWVIMWNI'

In [152]:
token_data = get_access_token(authorization_code, code_verifier)

if token_data:
    access_token = token_data["access_token"]
    refresh_token = token_data["refresh_token"]
    print(f"Token de acceso: {access_token}")
    print(f"Refresh token: {refresh_token}")

    # 5. Usar el access token para hacer solicitudes a la API de Kick
    # ...
    response = requests.get(
        "https://api.kick.com/public/v1/categories?q=ga%",
        headers={"Authorization":authorization_code},
        
    )
    data = response.json()
    print(data)

    # 6. Refrescar el token (cuando expire)
    new_token_data = refresh_access_token(refresh_token)
    if new_token_data:
        print("Token refrescado correctamente.")

    # 7. Revocar un token (cuando sea necesario)
    revoke_token(access_token, "access_token") # Ejemplo: revocar el access token
    revoke_token(refresh_token, "refresh_token") # Ejemplo: revocar el refresh token

else:
    print("No se pudo obtener el token.")

Token de acceso: N2FLZGFLMGMTOGU5MY0ZNDAZLTGYNJITZGQYZWQ2ZJCYOTEY
Refresh token: NTLKMMFJMZUTZDIZNS01ODLKLTGXNZITYWRJZJQ3YJJLOTGW
{'data': {}, 'message': 'Unauthorized'}
Token refrescado correctamente.
https://id.kick.com/oauth/revoke?token=N2FLZGFLMGMTOGU5MY0ZNDAZLTGYNJITZGQYZWQ2ZJCYOTEY&token_hint_type=access_token
Error al revocar el token: 404 - 
https://id.kick.com/oauth/revoke?token=NTLKMMFJMZUTZDIZNS01ODLKLTGXNZITYWRJZJQ3YJJLOTGW&token_hint_type=refresh_token
Error al revocar el token: 404 - 


In [ ]:
response = requests.get(
    "https://api.kick.com/public/v1/categories?q=",
    headers={"Authorization":access_token},
)
data = response.json()
print(data)

{'data': {}, 'message': 'Unauthorized'}


In [66]:
data

{'data': {}, 'message': 'Unauthorized'}

In [150]:
token_data